In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Cristóvão Beirão da Cruz e Silva <cbeiraod@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

#############################################################################
# Modified for ETROC2 I2C testing in jupyter notebooks, Murtaza Safdari
#############################################################################

# Basic Configuration

## Imports

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import logging
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time
import datetime
from tqdm import tqdm
import os, sys
import multiprocessing
os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
import run_script
import importlib
importlib.reload(run_script)
import pickle
import pandas
from pathlib import Path
import subprocess
from scipy.optimize import curve_fit
from scipy.stats import crystalball, chisquare
import sqlite3
from math import ceil
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from fnmatch import fnmatch
import hist
from hist import Hist
import matplotlib.cm as cm
import scipy.stats as stats
from fnmatch import fnmatch
import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
from i2c_gui.chips.etroc2_chip import register_decoding
from notebooks.notebook_helpers import *

## Specify Global Info

In [ ]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data
chip_name = "ET2-W42-IP7-2"
tag = "HV210V"
chip_figname = f"{chip_name}_LowBiasCurrent_{tag}"
chip_figtitle= chip_name+" LowBiasCurrent "+tag

# 'The port name the USB-ISS module is connected to. Default: COM3'
port = "/dev/ttyACM0"
# I2C addresses for the pixel block and WS
chip_address = 0x62
ws_address = None

fig_outdir = Path('../ETROC-figures')
fig_outdir = fig_outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

hostname = '192.168.2.7'

## Make i2c_connection class object for chip configuration

In [ ]:
i2c_conn = i2c_connection(port,[chip_address],[chip_name],[("1","1"),("1","1")])

## Config chips

In [ ]:
# 0-0-0-disable/default all pixels - auto TH CAL - set basic peripherals - peri reg check - pixel ID check
i2c_conn.config_chips('00011111')

### Visualize the learned Baselines (BL) and Noise Widths (NW)

Note that the NW represents the full width on either side of the BL

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
BL_map_THCal,NW_map_THCal,BL_df = i2c_conn.get_auto_cal_maps(chip_address)
fig = plt.figure(dpi=200, figsize=(10,5))
gs = fig.add_gridspec(1,2)

ax0 = fig.add_subplot(gs[0,0])
ax0.set_title(f"{chip_figtitle}: BL (DAC LSB)", size=8)
img0 = ax0.imshow(BL_map_THCal, interpolation='none')
ax0.set_aspect("equal")
ax0.invert_xaxis()
ax0.invert_yaxis()
plt.xticks(range(16), range(16), rotation="vertical")
plt.yticks(range(16), range(16))
divider = make_axes_locatable(ax0)
cax = divider.append_axes('right', size="5%", pad=0.05)
fig.colorbar(img0, cax=cax, orientation="vertical")

ax1 = fig.add_subplot(gs[0,1])
ax1.set_title(f"{chip_figtitle}: NW (DAC LSB)", size=8)
img1 = ax1.imshow(NW_map_THCal, interpolation='none')
ax1.set_aspect("equal")
ax1.invert_xaxis()
ax1.invert_yaxis()
plt.xticks(range(16), range(16), rotation="vertical")
plt.yticks(range(16), range(16))
divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size="5%", pad=0.05)
fig.colorbar(img1, cax=cax, orientation="vertical")

for x in range(16):
    for y in range(16):
        # if(BL_map_THCal.T[x,y]==0): continue
        ax0.text(x,y,f"{BL_map_THCal.T[x,y]:.0f}", c="white", size=5, rotation=45, fontweight="bold", ha="center", va="center")
        ax1.text(x,y,f"{NW_map_THCal.T[x,y]:.0f}", c="white", size=5, rotation=45, fontweight="bold", ha="center", va="center")
# plt.savefig(fig_path+"/"+chip_figname+"_BL_NW_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
plt.show()

## One time run to set fpga firmware and LEDs

In [ ]:
parser = run_script.getOptionParser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 20 -o CanBeRemoved -v -w -s 0x000C -p 0x000f -d 0x0800 --clear_fifo".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_Start_LEDs'))
process.start()

IPC_queue.put('start L1A trigger bit')
while not IPC_queue.empty():
    pass
time.sleep(10)
IPC_queue.put('stop DAQ')
IPC_queue.put('stop L1A trigger bit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

## Define charges

In [ ]:
QInjEns = [5, 15, 25]

## Specify pixels for scan

In [ ]:
# row_list = [15, 15, 15, 15, 14, 14, 14, 14]
# col_list = [6, 7, 8, 9, 6, 7, 8, 9]
row_list = [15, 14]
col_list = [6, 6]
scan_list = list(zip(col_list, row_list))
print(scan_list)

# Peak DAC Value Binary Search

In [ ]:
DAC_scan_max = 1020
scan_name = chip_figname+"_VRef_SCurve_BinarySearch"
fpga_time = 3

today = datetime.date.today()
todaystr = "../ETROC-Data/" + today.isoformat() + "_Array_Test_Results/"
base_dir = Path(todaystr)
base_dir.mkdir(exist_ok=True)

# Loop for enable/disable charge injection per pixel (single!!!)
for index, (col, row) in zip(range(len(row_list)), scan_list):
    print("Pixel:",col,row)
    pixel_connected_chip = i2c_conn.get_pixel_chip(chip_address, row, col)
    # Enable charge injection
    i2c_conn.pixel_decoded_register_write("disDataReadout", "0", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("QInjEn", "1", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("disTrigPath", "0", pixel_connected_chip)
    # Bypass Cal Threshold
    i2c_conn.pixel_decoded_register_write("Bypass_THCal", "1", pixel_connected_chip)
    for QInj in QInjEns:
        # Modifying charge injected
        i2c_conn.pixel_decoded_register_write("QSel", format(QInj, '05b'), pixel_connected_chip)
        threshold_name = scan_name+f'_Pixel_C{col}_R{row}_QInj_{QInj}'
        (options, args) = parser.parse_args(args=f"--useIPC --hostname {hostname} -o {threshold_name} -v -w --reset_till_trigger_linked -s 0x000C -p 0x000f -d 0x0800 -c 0x0001 --fpga_data_time_limit 3 --fpga_data_QInj --check_trigger_link_at_end --nodaq --clear_fifo".split())
        IPC_queue = multiprocessing.Queue()
        process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_link'))
        process.start()
        process.join()

        a = BL_map_THCal[row][col]
        b = DAC_scan_max
        header_max = -1
        while b-a>1:
            DAC = int(np.floor((a+b)/2))
            # Set the DAC v, Qinj {Qinj}fCalue to the value being scanned
            i2c_conn.pixel_decoded_register_write("DAC", format(DAC, '010b'), pixel_connected_chip)
            # print("QInj:", QInj, ", DAC:", DAC, ", Read TH:", i2c_conn.pixel_decoded_register_read("TH", "Status", pixel_connected_chip, need_int=True))

            (options, args) = parser.parse_args(args=f"--useIPC --hostname {hostname} -o {threshold_name} -v --reset_till_trigger_linked -s 0x000C -p 0x000f -d 0x0800 -c 0x0001 --fpga_data_time_limit {int(fpga_time)} --fpga_data_QInj --check_trigger_link_at_end --nodaq --DAC_Val {int(DAC)}".split())
            IPC_queue = multiprocessing.Queue()
            process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_{QInj}_{DAC}'))
            process.start()
            process.join()
            
            continue_flag = False
            root = '../ETROC-Data'
            file_pattern = "*FPGA_Data.dat"
            path_pattern = f"*{today.isoformat()}_Array_Test_Results/{threshold_name}"
            file_list = []
            for path, subdirs, files in os.walk(root):
                if not fnmatch(path, path_pattern): continue
                for name in files:
                    pass
                    if fnmatch(name, file_pattern):
                        file_list.append(os.path.join(path, name))
            total_files = len(file_list)
            for file_index, file_name in enumerate(file_list):
                with open(file_name) as infile:
                    lines = infile.readlines()
                    last_line = lines[-1]
                    first_line = lines[0]
                    header_max = int(first_line.split(',')[4])
                    text_list = last_line.split(',')
                    FPGA_state = text_list[0]
                    line_DAC = int(text_list[-1])
                    if(FPGA_state==0 or line_DAC!=DAC): 
                        continue_flag=True
                        continue
                    TDC_data = int(text_list[3])
                    # Condition handling for Binary Search
                    if(TDC_data>=header_max/2.):
                        a = DAC
                    else:
                        b = DAC                    
            if(continue_flag): continue  
            
    # Disable charge injection
    i2c_conn.pixel_decoded_register_write("QInjEn", "0", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("disDataReadout", "1", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("disTrigPath", "1", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("DAC", format(0x3ff, '010b'), pixel_connected_chip)

### Extract Peak DAC

In [ ]:
scan_name = chip_figname+"_VRef_SCurve_BinarySearch"
QInj_Peak_DAC_map = {row:{col:{q:0 for q in QInjEns} for col in range(16)} for row in range(16)}
for index, (col, row) in zip(range(len(row_list)), scan_list): 
    print("Pixel:",col,row)
    for QInj in tqdm(QInjEns, desc=f'Charge Loop for Pixel {col},{row}', leave=False):
        threshold_name = scan_name+f'_Pixel_C{col}_R{row}_QInj_{QInj}'
        path_pattern = f"*{today.isoformat()}_Array_Test_Results/{threshold_name}"
        file_list = []
        for path, subdirs, files in os.walk(root):
            if not fnmatch(path, path_pattern): continue
            for name in files:
                pass
                if fnmatch(name, file_pattern):
                    file_list.append(os.path.join(path, name))
        total_files = len(file_list)
        for file_index, file_name in enumerate(file_list):
            with open(file_name) as infile:
                last_line = infile.readlines()[-1]
                text_list = last_line.split(',')
                DAC = int(text_list[-1])
                QInj_Peak_DAC_map[row][col][QInj] = DAC

# DAQ

## Define DAQ Running Function

In [ ]:
def run_daq(timePerPixel, deadTime, triggerBitDelay, dirname):
    
    time_per_pixel = timePerPixel
    dead_time_per_pixel = deadTime
    trigger_bit_delay = triggerBitDelay
    total_scan_time = time_per_pixel + dead_time_per_pixel
    outname = dirname

    today = datetime.date.today()
    todaystr = "../ETROC-Data/" + today.isoformat() + "_Array_Test_Results/"
    base_dir = Path(todaystr)
    base_dir.mkdir(exist_ok=True)

    parser = run_script.getOptionParser() 
    (options, args) = parser.parse_args(args=f"--useIPC --hostname {hostname} -t {int(total_scan_time)} -o {outname} -v -w -s 0x000C -p 0x000f -d 0x0800 --clear_fifo --reset_till_trigger_linked".split())
    IPC_queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process'))
    process.start()

    IPC_queue.put('start L1A trigger bit')
    while not IPC_queue.empty():
        pass

    # delay = '000011'+format(trigger_bit_delay, '010b')
    # hex_delay = hex(int(delay, base=2))
    # IPC_queue.put(f'change delay {hex_delay}') 
    # while not IPC_queue.empty():
    #     pass

    time.sleep(time_per_pixel)
    IPC_queue.put('stop L1A trigger bit')

    time.sleep(1)
    IPC_queue.put('stop DAQ')
    while not IPC_queue.empty():
        pass

    IPC_queue.put('allow threads to exit')

    process.join()

## Run DAQ For Each Charge & DAC

In [ ]:
pedestal_scan_step = 20
scan_name = chip_figname+"_VRef_TDC_DAQ"
fpga_time = 3

today = datetime.date.today()
todaystr = "../ETROC-Data/" + today.isoformat() + "_Array_Test_Results/"
base_dir = Path(todaystr)
base_dir.mkdir(exist_ok=True)

col_list, row_list = zip(*scan_list)

# Loop for enable/disable charge injection per pixel (single!!!)
for index, (col, row) in zip(range(len(row_list)), scan_list):
    print("Pixel:",col,row)
    pixel_connected_chip = i2c_conn.get_pixel_chip(chip_address, row, col)
    # Enable charge injection
    i2c_conn.pixel_decoded_register_write("disDataReadout", "0", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("QInjEn", "1", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("disTrigPath", "0", pixel_connected_chip)
    # Bypass Cal Threshold
    i2c_conn.pixel_decoded_register_write("Bypass_THCal", "1", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("L1Adelay", format(0x01f5, '09b'), pixel_connected_chip) # Change L1A delay - circular buffer in ETROC2 pixel

    for QInj in QInjEns:
        # Modifying charge injected
        i2c_conn.pixel_decoded_register_write("QSel", format(QInj, '05b'), pixel_connected_chip)
        thresholds = np.arange(BL_map_THCal[row][col]-20, QInj_Peak_DAC_map[row][col][QInj]+10, pedestal_scan_step)
        
        for DAC in tqdm(thresholds, desc=f'DAC Loop for Pixel ({col},{row}) & Charge {QInj} fC', leave=False):
            # Set the DAC v, Qinj {Qinj}fCalue to the value being scanned
            i2c_conn.pixel_decoded_register_write("DAC", format(int(DAC), '010b'), pixel_connected_chip)
            # TH = i2c_conn.pixel_decoded_register_read("TH", "Status", pixel_connected_chip, need_int=True)
            threshold_name = scan_name+f'_Pixel_C{col}_R{row}_QInj_{QInj}_Threshold_{int(DAC)}'
            run_daq(5, 3, 485, threshold_name)
            
    # Disable charge injection
    i2c_conn.pixel_decoded_register_write("QInjEn", "0", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("disDataReadout", "1", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("disTrigPath", "1", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("DAC", format(0x3ff, '010b'), pixel_connected_chip)


# Plotting

## Define Helper Functions

In [ ]:
# def turning_point(key_array, array):
#     return np.argmax(np.where(key_array>2000, array, 0))
def return_empty_list():
    return {(row,col,q):{} for q in QInjEns for col,row in scan_list}

In [ ]:
scan_name = chip_figname+"_VRef_TDC_DAQ"
root = '../ETROC-Data'
file_pattern = "*translated*.dat"
path_pattern = f"*{scan_name}*"
file_list = []
for path, subdirs, files in os.walk(root):
    if not fnmatch(path, path_pattern): continue
    for name in files:
        pass
        if fnmatch(name, file_pattern):
            file_list.append(os.path.join(path, name))
            print(file_list[-1])

In [ ]:
hit_counts = return_empty_list()
CAL_sum = return_empty_list()
CAL_sum_sq = return_empty_list()
TOA_sum = return_empty_list()
TOA_sum_sq = return_empty_list()
TOT_sum = return_empty_list()
TOT_sum_sq = return_empty_list()
CAL_mean = return_empty_list()
CAL_std = return_empty_list()
TOA_mean = return_empty_list()
TOA_std = return_empty_list()
TOT_mean = return_empty_list()
TOT_std = return_empty_list()

In [ ]:
total_files = len(file_list)
for file_index, file_name in enumerate(file_list):
    col = int(file_name.split('/')[-2].split('_')[-6][1:])
    row = int(file_name.split('/')[-2].split('_')[-5][1:])
    QInj = int(file_name.split('/')[-2].split('_')[-3])
    DAC = int(file_name.split('/')[-2].split('_')[-1])
    hit_counts[row, col, QInj][DAC] = 0
    CAL_sum[row, col, QInj][DAC] = 0
    CAL_sum_sq[row, col, QInj][DAC] = 0
    TOA_sum[row, col, QInj][DAC] = 0
    TOA_sum_sq[row, col, QInj][DAC] = 0
    TOT_sum[row, col, QInj][DAC] = 0
    TOT_sum_sq[row, col, QInj][DAC] = 0
    CAL_mean[row, col, QInj][DAC] = 0
    CAL_std[row, col, QInj][DAC] = 0
    TOA_mean[row, col, QInj][DAC] = 0
    TOA_std[row, col, QInj][DAC] = 0
    TOT_mean[row, col, QInj][DAC] = 0
    TOT_std[row, col, QInj][DAC] = 0
    with open(file_name) as infile:
        for line in infile:
            text_list = line.split()
            if text_list[2]=="HEADER":
                current_bcid = int(text_list[8])
            if text_list[2]=="TRAILER":
                previous_bcid = current_bcid
            if text_list[2]!="DATA": continue
            # col = int(text_list[6])
            # row = int(text_list[8])
            TOA = int(text_list[10])
            TOT = int(text_list[12])
            CAL = int(text_list[14])

            # if(CAL>200 or CAL<100): continue
            hit_counts[row, col, QInj][DAC] += 1 
            CAL_sum[row, col, QInj][DAC] += CAL
            CAL_sum_sq[row, col, QInj][DAC] += CAL*CAL
            TOA_sum[row, col, QInj][DAC] += TOA
            TOA_sum_sq[row, col, QInj][DAC] += TOA*TOA
            TOT_sum[row, col, QInj][DAC] += TOT
            TOT_sum_sq[row, col, QInj][DAC] += TOT*TOT

In [ ]:
for row, col, QInj in hit_counts:
    for DAC in hit_counts[row, col, QInj]:
        if(hit_counts[row, col, QInj][DAC]==0):
            CAL_mean[row, col, QInj].pop(DAC)
            CAL_std[row, col, QInj].pop(DAC)
            TOA_mean[row, col, QInj].pop(DAC)
            TOA_std[row, col, QInj].pop(DAC)
            TOT_mean[row, col, QInj].pop(DAC)
            TOT_std[row, col, QInj].pop(DAC)
            continue
        CAL_mean[row, col, QInj][DAC] = CAL_sum[row, col, QInj][DAC]/hit_counts[row, col, QInj][DAC]
        CAL_std[row, col, QInj][DAC] = np.sqrt((CAL_sum_sq[row, col, QInj][DAC]/hit_counts[row, col, QInj][DAC]) - pow(CAL_mean[row, col, QInj][DAC], 2))
        TOA_mean[row, col, QInj][DAC] = TOA_sum[row, col, QInj][DAC]/hit_counts[row, col, QInj][DAC]
        TOA_std[row, col, QInj][DAC] = np.sqrt((TOA_sum_sq[row, col, QInj][DAC]/hit_counts[row, col, QInj][DAC]) - pow(TOA_mean[row, col, QInj][DAC], 2))
        TOT_mean[row, col, QInj][DAC] = TOT_sum[row, col, QInj][DAC]/hit_counts[row, col, QInj][DAC]
        TOT_std[row, col, QInj][DAC] = np.sqrt((TOT_sum_sq[row, col, QInj][DAC]/hit_counts[row, col, QInj][DAC]) - pow(TOT_mean[row, col, QInj][DAC], 2))

## TOA TOT CAL plots vs DAC

In [ ]:
colors = [plt.cm.viridis(i) for i in np.linspace(0,1,len(QInjEns))]
col_list, row_list = zip(*scan_list)
u_cl = np.sort(np.unique(col_list))
u_rl = np.sort(np.unique(row_list))
fig = plt.figure(dpi=200, figsize=(len(np.unique(u_cl))*5,len(np.unique(u_rl))*5))
gs = fig.add_gridspec(len(np.unique(u_rl)),len(np.unique(u_cl)))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        for i, QInj in enumerate(QInjEns[0:]):
            ax0.plot(TOA_mean[row, col, QInj].keys(), np.array(list(TOA_mean[row, col, QInj].values())), '.-', label=f"{QInj} fC", color=colors[i],lw=0.5,markersize=2)

            # p = np.polyfit(TOA_mean[row, col, QInj].keys(), np.array(list(TOA_mean[row, col, QInj].values())), 3)
            # x_range = np.linspace(248, TOA_mean[row, col, QInj].keys(), 100)
            # Yest = np.polyval(p, x_range)
            # ax0.plot(x_range, Yest, 'k-', lw=0.7)

        ax0.set_xlabel("DAC Value [LSB]")
        ax0.set_ylabel("TOA Mean [LSB]")
        plt.grid()
        plt.title(f"{chip_figtitle}, Pixel ({row},{col}) QInj S-Curve",size=9)
        plt.legend(loc="upper right")

        
plt.tight_layout()
# plt.savefig("figures/toa_mean_scurve.pdf")
plt.show()

In [ ]:
colors = [plt.cm.viridis(i) for i in np.linspace(0,1,len(QInjEns))]
col_list, row_list = zip(*scan_list)
u_cl = np.sort(np.unique(col_list))
u_rl = np.sort(np.unique(row_list))
fig = plt.figure(dpi=200, figsize=(len(np.unique(u_cl))*5,len(np.unique(u_rl))*5))
gs = fig.add_gridspec(len(np.unique(u_rl)),len(np.unique(u_cl)))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        for i, QInj in enumerate(QInjEns[0:]):
            ax0.plot(TOA_std[row, col, QInj].keys(), np.array(list(TOA_std[row, col, QInj].values())), '.-', label=f"{QInj} fC", color=colors[i],lw=0.5,markersize=2)
        ax0.set_xlabel("DAC Value [LSB]")
        ax0.set_ylabel("TOA Std [LSB]")
        plt.title(f"{chip_figtitle}, Pixel ({row},{col}) QInj S-Curve",size=9)
        plt.legend(loc="upper right")
plt.tight_layout()
# plt.savefig("figures/toa_std_scurve.pdf")
plt.show()

In [ ]:
colors = [plt.cm.viridis(i) for i in np.linspace(0,1,len(QInjEns))]
col_list, row_list = zip(*scan_list)
u_cl = np.sort(np.unique(col_list))
u_rl = np.sort(np.unique(row_list))
fig = plt.figure(dpi=200, figsize=(len(np.unique(u_cl))*5,len(np.unique(u_rl))*5))
gs = fig.add_gridspec(len(np.unique(u_rl)),len(np.unique(u_cl)))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        for i, QInj in enumerate(QInjEns[0:]):
            ax0.plot(TOT_mean[row, col, QInj].keys(), np.array(list(TOT_mean[row, col, QInj].values())), '.-', label=f"{QInj} fC", color=colors[i],lw=0.5,markersize=2)
        ax0.set_xlabel("DAC Value [LSB]")
        ax0.set_ylabel("TOT mean [LSB]")
        plt.title(f"{chip_figtitle}, Pixel ({row},{col}) QInj S-Curve",size=9)
        plt.legend(loc="upper right")
plt.tight_layout()
# plt.savefig("figures/tot_mean_scurve.pdf")
plt.show()

In [ ]:
colors = [plt.cm.viridis(i) for i in np.linspace(0,1,len(QInjEns))]
col_list, row_list = zip(*scan_list)
u_cl = np.sort(np.unique(col_list))
u_rl = np.sort(np.unique(row_list))
fig = plt.figure(dpi=200, figsize=(len(np.unique(u_cl))*5,len(np.unique(u_rl))*5))
gs = fig.add_gridspec(len(np.unique(u_rl)),len(np.unique(u_cl)))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        for i, QInj in enumerate(QInjEns[0:]):
            ax0.plot(TOT_std[row, col, QInj].keys(), np.array(list(TOT_std[row, col, QInj].values())), '.-', label=f"{QInj} fC", color=colors[i],lw=0.5,markersize=2)
        ax0.set_xlabel("DAC Value [LSB]")
        ax0.set_ylabel("TOT Std [LSB]")
        plt.title(f"{chip_figtitle}, Pixel ({row},{col}) QInj S-Curve",size=9)
        plt.legend(loc="upper right")
plt.tight_layout()
# plt.savefig("figures/tot_std_scurve.pdf")
plt.show()

In [ ]:
colors = [plt.cm.viridis(i) for i in np.linspace(0,1,len(QInjEns))]
col_list, row_list = zip(*scan_list)
u_cl = np.sort(np.unique(col_list))
u_rl = np.sort(np.unique(row_list))
fig = plt.figure(dpi=200, figsize=(len(np.unique(u_cl))*5,len(np.unique(u_rl))*5))
gs = fig.add_gridspec(len(np.unique(u_rl)),len(np.unique(u_cl)))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        for i, QInj in enumerate(QInjEns[0:]):
            ax0.plot(CAL_mean[row, col, QInj].keys(), np.array(list(CAL_mean[row, col, QInj].values())), '.-', label=f"{QInj} fC", color=colors[i],lw=0.5,markersize=2)
        ax0.set_xlabel("DAC Value [LSB]")
        ax0.set_ylabel("CAL Mean [LSB]")
        plt.title(f"{chip_figtitle}, Pixel ({row},{col}) QInj S-Curve",size=9)
        plt.legend(loc="upper right")
plt.tight_layout()
# plt.savefig("figures/cal_mean_scurve.pdf")
plt.show()

In [ ]:
colors = [plt.cm.viridis(i) for i in np.linspace(0,1,len(QInjEns))]
col_list, row_list = zip(*scan_list)
u_cl = np.sort(np.unique(col_list))
u_rl = np.sort(np.unique(row_list))
fig = plt.figure(dpi=200, figsize=(len(np.unique(u_cl))*5,len(np.unique(u_rl))*5))
gs = fig.add_gridspec(len(np.unique(u_rl)),len(np.unique(u_cl)))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        for i, QInj in enumerate(QInjEns[0:]):
            ax0.plot(CAL_std[row, col, QInj].keys(), np.array(list(CAL_std[row, col, QInj].values())), '.-', label=f"{QInj} fC", color=colors[i],lw=0.5,markersize=2)
        ax0.set_xlabel("DAC Value [LSB]")
        ax0.set_ylabel("CAL Std [LSB]")
        plt.title(f"{chip_figtitle}, Pixel ({row},{col}) QInj S-Curve",size=9)
        plt.legend(loc="upper right")
plt.tight_layout()
# plt.savefig("figures/cal_std_scurve.pdf")
plt.show()

# TOA vs TOT

In [ ]:
fig = plt.figure(dpi=200, figsize=(8,4))
gs = fig.add_gridspec(len(scan_list),len(scan_list))
u_cl = np.sort(np.unique(col_list))
u_rl = np.sort(np.unique(row_list))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        for QInj in QInjEns:
            ax0.plot(TOA_mean[row, col, QInj].keys(), np.array(list(TOT_mean[row, col, QInj].values())), '.-', label=f"{QInj} fC", color=colors[i],lw=0.5,markersize=2)
        ax0.set_xlabel("TOA Mean [LSB]")
        ax0.set_ylabel("TOT Mean [LSB]")
        plt.title(f"{chip_figtitle}, Pixel ({row},{col}) QInj S-Curve",size=9)
        plt.legend(loc="upper right")
plt.tight_layout()
plt.savefig("figures/toa_mean_vs_tot_mean_scurve.pdf")
plt.show()

In [ ]:
fig = plt.figure(dpi=100, figsize=(8,8))
gs = fig.add_gridspec(1,1)
row = 3
col = 3
QInj = 30
ax0 = fig.add_subplot(gs[0,0])
tp_idx = turning_point(np.array(list(hit_counts[(row,col)][QInj].values())), thresholds[QInj])
print(tp_idx)
ax0.plot(np.array(list(TOA_mean[(row,col)][QInj].values()))[15:tp_idx-0], np.array(list(TOT_mean[(row,col)][QInj].values()))[15:tp_idx-0], '.-', label=f"{QInj} fC")
ax0.plot(toa_30fc, tot_30fc, '.')
p, cov = np.polyfit(tot_30fc, toa_30fc, 3, cov = True)
n = tot_30fc.size
x_range = np.linspace(10, 80, 100)
Yest = np.polyval(p, x_range)
ax0.plot(Yest, x_range, 'k-', label=f'TOA = {p[-1]:.3f} + {p[-2]:.3f}.TOT + {p[-3]:.3f}.TOT**2 + {p[-4]:.3f}.TOT**3')
        
ax0.set_xlabel("TOA Mean [LSB]")
ax0.set_ylabel("TOT Mean [LSB]")
plt.title(f"Bare Chip 01E2 #52, Pixel ({row},{col}), Qinj S-Curve, PA Low Bias Current Mode")
plt.legend(loc="upper left")
plt.tight_layout()
# plt.savefig("figures/toa_vs_tot_scurve.pdf")
plt.show()

In [ ]:
fig = plt.figure(dpi=100, figsize=(8,8))
gs = fig.add_gridspec(1,1)
row = 3
col = 3
QInj = 30
ax0 = fig.add_subplot(gs[0,0])
code_conversion = 3.125/np.array(cal_30fc)
ax0.plot(code_conversion*toa_30fc, code_conversion*tot_30fc, '.')
p, cov = np.polyfit(code_conversion*tot_30fc, code_conversion*toa_30fc, 3, cov = True)
n = tot_30fc.size
x_range = np.linspace(0.3, 1.5, 100)
Yest = np.polyval(p, x_range)
ax0.plot(Yest, x_range, 'k-', label=f'TOA = {p[-1]:.3f} + {p[-2]:.3f}.TOT + {p[-3]:.3f}.TOT**2 + {p[-4]:.3f}.TOT**3')
ax0.set_xlabel("TOA [ns]")
ax0.set_ylabel("TOT [ns]")
plt.title(f"Bare Chip 01E2 #52, Pixel ({row},{col}), Qinj S-Curve, PA Low Bias Current Mode")
plt.legend(loc="upper left")
plt.tight_layout()
plt.savefig("figures/toa_vs_tot_scurve.pdf")
plt.show()

In [ ]:
fig = plt.figure(dpi=100, figsize=(8,8))
gs = fig.add_gridspec(1,1)
row = 3
col = 3
QInj = 30
ax = fig.add_subplot(gs[0,0])
Y_resid = 1000*(code_conversion*toa_30fc - np.polyval(p, code_conversion*tot_30fc))
hist_1 = hist.Hist(
    hist.axis.Regular(
        8, -50, 50, name="TOA", label="TOA Corrected [ps]", underflow=False, overflow=False
    )
).fill(Y_resid)
hist_1.project("TOA").plot1d(ax=ax, lw=1, histtype="fill")
ax.set_xticks(ax.get_xticks(),ax.get_xticklabels())
plt.title(f"Bare Chip 01E2 #52, Pixel ({row},{col}), Qinj S-Curve, PA Low Bias Current Mode")
plt.tight_layout()
# plt.savefig("figures/toa_vs_tot_scurve.pdf")
plt.show()

In [ ]:
Y_resid = 1000*(code_conversion*toa_30fc - np.polyval(p, code_conversion*tot_30fc))
hist_1 = hist.Hist(
    hist.axis.Regular(
        8, -50, 50, name="TOA", label="TOA Corrected [ps]", underflow=False, overflow=False
    )
).fill(Y_resid)
fig = plt.figure(figsize=(10, 8))
# hist_1.plot_pull("constant*np.exp(-(x-mean)**2. / (2*sigma**2))",
#                 fit_fmt="")
# hist_1.plot_pull("gaus",
#                 fit_fmt=r"{name} = {value:.3g} $\pm$ {error:.3g}")
main_ax_artists, sublot_ax_arists = hist_1.plot_pull(
    "normal",
    fit_fmt=r"{name} = {value:.3g} $\pm$ {error:.3g}",
    eb_ecolor="steelblue",
    eb_mfc="steelblue",
    eb_mec="steelblue",
    eb_fmt="o",
    eb_ms=6,
    eb_capsize=1,
    eb_capthick=2,
    eb_alpha=0.8,
    fp_c="hotpink",
    fp_ls="-",
    fp_lw=2,
    fp_alpha=0.8,
    bar_fc="royalblue",
    pp_num=3,
    pp_fc="royalblue",
    pp_alpha=0.618,
    pp_ec=None,
    ub_alpha=0.2,
)

In [ ]:
fig = plt.figure(dpi=100, figsize=(6,6))
gs = fig.add_gridspec(len(scan_list),len(scan_list))
u_cl = np.sort(np.unique(col_list))
u_rl = np.sort(np.unique(row_list))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        QInj = 30
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        tp_idx = turning_point(np.array(list(hit_counts[(row,col)][QInj].values())), thresholds[QInj])
        ax0.plot(np.array(list(TOA_mean[(row,col)][QInj].values()))[15:tp_idx-0], np.array(list(CAL_mean[(row,col)][QInj].values()))[15:tp_idx-0], '.-', label=f"{QInj} fC")
        ax0.plot(toa_30fc, cal_30fc, '.')
        ax0.set_xlabel("TOA Mean [LSB]")
        ax0.set_ylabel("CAL Mean [LSB]")
        plt.title(f"Bare Chip 01E2 #52, Pixel ({row},{col}), Qinj S-Curve, PA Low Bias Current Mode")
        plt.legend(loc="upper right")
plt.tight_layout()
plt.savefig("figures/toa_vs_cal_scurve.pdf")
plt.show()

In [ ]:
fig = plt.figure(dpi=100, figsize=(6,6))
gs = fig.add_gridspec(len(scan_list),len(scan_list))
u_cl = np.sort(np.unique(col_list))
u_rl = np.sort(np.unique(row_list))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        QInj = 30
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        tp_idx = turning_point(np.array(list(hit_counts[(row,col)][QInj].values())), thresholds[QInj])
        ax0.plot(np.array(list(TOT_mean[(row,col)][QInj].values()))[15:tp_idx-0], np.array(list(CAL_mean[(row,col)][QInj].values()))[15:tp_idx-0], '.-', label=f"{QInj} fC")
        ax0.plot(tot_30fc, cal_30fc, '.')
        ax0.set_xlabel("TOT Mean [LSB]")
        ax0.set_ylabel("CAL Mean [LSB]")
        plt.title(f"Bare Chip 01E2 #52, Pixel ({row},{col}), Qinj S-Curve, PA Low Bias Current Mode")
        plt.legend(loc="upper right")
plt.tight_layout()
plt.savefig("figures/tot_vs_cal_scurve.pdf")
plt.show()

In [ ]:
del i2c_conn